In [1]:
import os
print "current dir: ", os.getcwd()
os.chdir("/home/foodmap/food101/")
print "current dir: ", os.getcwd()

current dir:  /home/foodmap/food101/notebooks
current dir:  /home/foodmap/food101


In [2]:
from location.locations import Cities, Countries
from twitter.Tweet import Tweet
from location.get_location_from_tweet import *
import random
from index import preprocess_tweet
import codecs, json

citiesIndex, citiesInfo = Cities.loadFromFile()
countriesIndex, countriesInfo = Countries.loadFromFile()
ccDict = Countries.countryCodeDict(countriesInfo)

All cities with all name:  22309
All cities unique geonameid:   23516
All countries with all names:  24782
All countries unique geonameid:   251
Len of country code dict 251
All cities with all name:  22309
All cities unique geonameid:   23516
All countries with all names:  24782
All countries unique geonameid:   251
Len of country code dict 251


In [3]:
inputFile = "/home/foodmap/data/food-tweets/food-tweets-2017-06-15.json.gz"

In [ ]:
def get_image_category(img_url):
    """
    Gets the food category for a given img url.
    :param img_url:
    :return:
    """
    request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
    res = requests.get(request_string + img_url)
    try:
        candidates = json.loads(res.text)["guessed"]
    except:
        print "Error! Couldn't parse json ", res.text
        return None
    if len(candidates) != 0:
        category = candidates[0]["label"]
    else:
        category = None
    return category

In [ ]:
tweetsAsDict = Tweet.getTweetAsDictionary(inputFile)
i = 0
countImg = 0
countIndexEntries = 0
countCategory = 0
countPlace = 0
countCoords = 0

# with codecs.open("locationresults-no-ambiguity.txt", "w", "utf-8") as f:
for tweet in tweetsAsDict:
    i+=1
    hasImg = False
    hasCategory = False
    hasCountry = False
    
    if i % 10 == 0:
        print "i ", i

    # 1. has photo?
    media_url = preprocess_tweet.get_media_url(tweet)
    if media_url is not None:
        hasImg = True
        countImg += 1
    else:
        continue

    # 2. has food category?
    if preprocess_tweet.get_image_category(media_url) is not None:
        hasCategory = True
        countCategory += 1
    else:
        continue
        
    if countCategory % 100 == 0:
        print "#cat ",countCategory

    # 3. has location? we need city and country; record 2 separate fields
    ## 3.1. check for place
    tweet_coords, tweet_place_city, tweet_place_country, tweet_place_country_code, user_loc = getLocationData(tweet)
    if tweet_coords is not None or tweet_place_country is not None:
        countCoords += 1

    #print tweet_coords, tweet_place_city, tweet_place_country, tweet_place_country_code
    ## 3.2. check for user location
    if (tweet_place_country is None):
        user_cities, user_countries = getUserLocation(user_loc, citiesIndex, citiesInfo, countriesIndex, countriesInfo)
        inferred_countries = inferCountryFromCity(user_cities, citiesIndex, citiesInfo, ccDict)
        city, country = getFinalUserLocation(user_cities, user_countries, inferred_countries)
    else:
        city = tweet_place_city
        country = tweet_place_country

#         if user_loc is not None:
#             f.write("User_loc:" + json.dumps(user_loc) + "\n")
#             f.write("If Place: " + json.dumps(tweet_place_city) + "\t" + json.dumps(tweet_place_country) + "\n")
#             f.write("Final Loc: " + json.dumps(city) + "\t" + json.dumps(country) + "\n")
#             f.write("\n")

    if (tweet_place_country is not None) or (country is not None):
        hasCountry =True
        countPlace += 1

    # SEE if CONDITIONS are met
    if hasImg and hasCategory and hasCountry:
        countIndexEntries +=1



print "Total relevant tweets in a day: ", i
print "Tweets with media", countImg
print "Tweets with category", countCategory
print "Tweets with place", countPlace
print "Tweets with coords", countCoords
print "Tweets in index", countIndexEntries

Opening file:  /home/foodmap/data/food-tweets/food-tweets-2017-06-15.json.gz
i  10
i  20
i  30
i  40
i  50
i  60
i  70
i  80
i  90
i  100
i  110
i  120
Error! Couldn't parse json  <!doctype html><html lang="en"><head><title>HTTP Status [500] – [Internal Server Error]</title><style type="text/css">h1 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:22px;} h2 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:16px;} h3 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:14px;} body {font-family:Tahoma,Arial,sans-serif;color:black;background-color:white;} b {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;} p {font-family:Tahoma,Arial,sans-serif;background:white;color:black;font-size:12px;} a {color:black;} a.name {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status [500] – [Internal Server Error]</h1><hr class=